In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from scipy.stats import uniform, randint

from tools import create_x_y

In [ ]:
df = pd.read_csv('../datasets/relevant/amzn.csv')

In [ ]:
VITAL_COLS = ["date_", "ticker", "close", "diffs", "bin_2", "bin_3", "bin_5"]
OTHER_COLS = [i for i in df.columns if i not in VITAL_COLS]
print(OTHER_COLS)

In [ ]:
def measure(x, y):
    clf = DecisionTreeClassifier(class_weight=weight_data(y))

    params = {
        'max_depth': [2, 3, 5, 10, 20, 50, 100, 200],
        'min_samples_leaf': [2, 3, 5, 10],
        'criterion': ["gini", "entropy"]
    }
    time_split = TimeSeriesSplit(n_splits=5)

    tree_search = GridSearchCV(
        clf,
        param_grid=params,
        cv=time_split,
        verbose=1,
        n_jobs=4,
    )
    split = int(0.8 * len(x))
    tree_search.fit(x[:split], y[:split])
    y_pred = tree_search.predict(x[split:])
    cm = confusion_matrix(y[split:], y_pred)
    acc = sum(y_pred == y[split:])/len(y_pred)
    
    return cm, acc

In [ ]:
acc = {}
cm = {}
for lag in [3, 6, 10, 16]:
    x, y = create_x_y(df, x_cols=OTHER_COLS, y_col="bin_3", lag=lag)
    cm[lag], acc[lag] = measure(x, y)

In [ ]:
acc

In [ ]:
cm